In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
! pip install --upgrade bitsandbytes evaluate
! pip install peft==0.12.0
! pip install -U transformers

/kaggle/input/w2v-nlp/testData.tsv
/kaggle/input/w2v-nlp/labeledTrainData.tsv
/kaggle/input/w2v-nlp/unlabeledTrainData.tsv
/kaggle/input/word2vec-nlp-tutorial/testData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/sampleSubmission.csv
/kaggle/input/word2vec-nlp-tutorial/unlabeledTrainData.tsv.zip
/kaggle/input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 77.5 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [2]:
import os
import sys
import logging
import datasets
import evaluate

import pandas as pd
import numpy as np

from transformers import AutoModelForSequenceClassification, DebertaV2Tokenizer, DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from peft import PromptEncoderConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
from sklearn.model_selection import train_test_split

train = pd.read_csv("/kaggle/input/w2v-nlp/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("/kaggle/input/w2v-nlp/testData.tsv", header=0, delimiter="\t", quoting=3)

if __name__ == '__main__':
    os.environ['WANDB_API_KEY'] = "a464ce6c3b972e3e7090ac20839b9a1daac1b608"
    
    train, val = train_test_split(train, test_size=.2)
    train_dict = {'label': train["sentiment"], 'text': train['review']}
    val_dict = {'label': val["sentiment"], 'text': val['review']}
    test_dict = {"text": test['review']}

    train_dataset = datasets.Dataset.from_dict(train_dict)
    val_dataset = datasets.Dataset.from_dict(val_dict)
    test_dataset = datasets.Dataset.from_dict(test_dict)

    batch_size = 8

    model_id = "microsoft/deberta-v3-xsmall"

    tokenizer = DebertaV2Tokenizer.from_pretrained(model_id)
    

    def preprocess_function(examples):
        return tokenizer(examples['text'], truncation=True)
    #使用 tokenizer 将文本转换为模型可以接受的格式。


    tokenized_train = train_dataset.map(preprocess_function, batched=True)
    tokenized_val = val_dataset.map(preprocess_function, batched=True)
    tokenized_test = test_dataset.map(preprocess_function, batched=True)
    # 数据预处理

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [3]:
 data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# 填充使适配模型


model = AutoModelForSequenceClassification.from_pretrained(model_id,
                                                           # quantization_config = BitsAndBytesConfig(load_in_8bit=True)
                                                          )
model = prepare_model_for_kbit_training(model)


peft_config = PromptEncoderConfig(
    num_virtual_tokens=20,
    encoder_hidden_size=128,
    task_type=TaskType.SEQ_CLS,
)


#LoRA 通过在特定层上插入低秩矩阵，减少了训练参数数量，优化了模型在资源受限环境下的训练表现。

# prepare int-8 model for training
# model = prepare_model_for_int8_training(model)



# model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, peft_config)
# You’re all set for training with whichever training method you prefer!



for param in model.parameters():
    param.requires_grad = True
#迁移学习中手动设置参数需要梯度    

    
# add LoRA adaptor
# model = get_peft_model(model, peft_config)
#LoRA 通过 get_peft_model 函数应用到预训练的 DebertaV2 模型上，并打印出可训练参数。

# model.print_trainable_parameters()

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


training_args = TrainingArguments(
    output_dir='./checkpoint',  # output directory
    num_train_epochs=3,  # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=4,  # batch size for evaluation
    warmup_steps=500,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
    logging_steps=100,
    save_strategy="no",
    evaluation_strategy="epoch",
)
# 设置训练参数


trainer = Trainer(
    model=model,  # the instantiated 🤗 Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=tokenized_train,  # training dataset
    eval_dataset=tokenized_val,  # evaluation dataset
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)
#使用 Trainer 进行训练和评估。
#训练参数通过 TrainingArguments 指定，其中包括批量大小、学习率、权重衰减和日志记录频率等。
#设置了 compute_metrics 函数来评估分类准确性。


trainer.train()
# 调用trainer.train()进行训练

prediction_outputs = trainer.predict(tokenized_test)
test_pred = np.argmax(prediction_outputs[0], axis=-1).flatten()
print(test_pred)

result_output = pd.DataFrame(data={"id": test["id"], "sentiment": test_pred})
os.mkdir("/kaggle/working/result")
result_output.to_csv("/kaggle/working/result/deberta_lora_int8.csv", index=False, quoting=3)
logging.info('result saved!')

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: skyfurynowonline (skyfurynowonline-yunnan-university). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241018_004313-zbsm5t6s
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./checkpoint
wandb: ⭐️ View project at https://wandb.ai/skyfurynowonline-yunnan-university/huggingface
wandb: 🚀 Vie

Epoch,Training Loss,Validation Loss,Accuracy
1,0.256000,0.346671,0.917400
2,0.461000,0.309461,0.938600
3,0.199100,0.381678,0.939800


[1 0 0 ... 0 1 1]
